In [1]:
from torchvision import models
import torch

dir(models)

['AlexNet',
 'AlexNet_Weights',
 'ConvNeXt',
 'ConvNeXt_Base_Weights',
 'ConvNeXt_Large_Weights',
 'ConvNeXt_Small_Weights',
 'ConvNeXt_Tiny_Weights',
 'DenseNet',
 'DenseNet121_Weights',
 'DenseNet161_Weights',
 'DenseNet169_Weights',
 'DenseNet201_Weights',
 'EfficientNet',
 'EfficientNet_B0_Weights',
 'EfficientNet_B1_Weights',
 'EfficientNet_B2_Weights',
 'EfficientNet_B3_Weights',
 'EfficientNet_B4_Weights',
 'EfficientNet_B5_Weights',
 'EfficientNet_B6_Weights',
 'EfficientNet_B7_Weights',
 'EfficientNet_V2_L_Weights',
 'EfficientNet_V2_M_Weights',
 'EfficientNet_V2_S_Weights',
 'GoogLeNet',
 'GoogLeNetOutputs',
 'GoogLeNet_Weights',
 'Inception3',
 'InceptionOutputs',
 'Inception_V3_Weights',
 'MNASNet',
 'MNASNet0_5_Weights',
 'MNASNet0_75_Weights',
 'MNASNet1_0_Weights',
 'MNASNet1_3_Weights',
 'MaxVit',
 'MaxVit_T_Weights',
 'MobileNetV2',
 'MobileNetV3',
 'MobileNet_V2_Weights',
 'MobileNet_V3_Large_Weights',
 'MobileNet_V3_Small_Weights',
 'RegNet',
 'RegNet_X_16GF_Weights'

In [2]:
model = models.vgg16(pretrained=True)
model.eval() # run the model with evaluation mode
# Why do we use : predefined model을 사용하기 때문에, eval을 사용한다. 
# eval을 통해 training이 아닌 불필요한 부분을 삭제하게 된다.

from torchsummary import summary
#summary(model, (3, 224, 224))

C:\Users\hanmu\AppData\Roaming\Python\Python39\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\hanmu\AppData\Roaming\Python\Python39\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
from torchvision import transforms

# 일반적으로, CNN의 test image size는 지정되어 있다
# 먼저, 256x256 사이즈로 받는다.
# crop : 224x224 사이즈로 줄인다.
# 최종적으로 input image size를 결정

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [11]:
# Download an example image from URL
filename = 'test_image.jpg'

import cv2 as cv

img = cv.imread(filename)

if img is None :
    print("Image load failed...!")
else :
    print("Image load success...!")



Image load success...!


In [12]:
from PIL import Image
from torchvision import transforms
input_image = Image.open(filename)

input_tensor = preprocess(input_image)  # Resize image (pre processing)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

'''    
    meaning of torch.no_grad()
    pytorch : auto grad engine이라는 것이 있다.
    auto grad engine : training code를 돌릴 때, gradient 값을 기억하고, 연산을 진행할 수 있도록 만들어두었음
    torch.no_grad : 사용되는 메모리를 줄일 수 있고, 연산량을 줄일 수 있다.
    전체적인 flow for lighter model : torch.eval --> torch.no_grad
    학습을 시키고자 한다면 : eval 모드가 아닌, model.train mode
'''

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
# print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
# softmax : change output to zero to one (probability check)
probabilities = torch.nn.functional.softmax(output[0], dim=0)
print(probabilities)

tensor([9.4343e-09, 1.0291e-07, 4.9967e-08, 5.6635e-09, 2.3764e-08, 7.8305e-09,
        2.0664e-09, 2.0004e-08, 1.9084e-08, 1.0984e-08, 3.9900e-09, 6.9910e-09,
        3.6025e-08, 1.0412e-08, 1.3839e-09, 7.2606e-09, 7.6493e-09, 8.5960e-10,
        4.8589e-09, 3.8308e-09, 7.9621e-10, 3.0739e-09, 4.5772e-09, 5.4077e-08,
        1.0546e-08, 1.0402e-08, 3.1916e-08, 1.5194e-08, 1.0427e-08, 6.4065e-09,
        1.4862e-08, 1.1093e-08, 4.2743e-08, 1.4851e-08, 1.0990e-08, 1.5056e-08,
        6.1443e-08, 1.0201e-08, 2.1768e-09, 6.1795e-09, 3.6831e-09, 2.2322e-09,
        8.1533e-09, 2.1666e-09, 4.5135e-09, 1.6371e-08, 1.7885e-09, 6.1511e-09,
        1.0484e-08, 2.5090e-08, 8.6920e-09, 1.5314e-07, 1.3317e-08, 1.5906e-08,
        1.6009e-08, 2.7659e-09, 1.5844e-08, 4.0847e-09, 1.6266e-08, 5.8565e-09,
        1.0424e-07, 8.3831e-09, 1.4986e-08, 7.0661e-09, 2.7973e-09, 5.0190e-09,
        1.9254e-08, 2.1324e-08, 1.1999e-08, 2.4543e-09, 1.1402e-08, 3.7060e-09,
        1.4168e-09, 2.3095e-08, 7.0681e-

In [29]:
# To store class txt file to working directory
import requests

url = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
response = requests.get(url)

with open("C:/Users/hanmu/Desktop/DLIP FILES/DeepLearning_TUTORIAL/4.Pytorch_TransferLearning/imagenet_classes.txt", "wb") as f:
    f.write(response.content)

In [30]:
# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

fire engine 0.8049407005310059
tow truck 0.15215733647346497
trailer truck 0.01319404598325491
garbage truck 0.009636634960770607
passenger car 0.00539995264261961
